<a href="https://colab.research.google.com/github/aikokass82-design/home-tasks-/blob/main/Lesson_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
from google.colab import drive
import pandas as pd
drive.mount("/content/drive")
from datetime import datetime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
advertising_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reclamni_zosobi-info.csv")
advertising_info

,id,issuedDate,distributorid,status,validFrom,validThrough,type,planesValue,horizontalSize,verticalSize,area,CATUTTC,district
0,63826-17,2017-10-30,33549639,Чинний,2017-10-30,2026-02-04,Спеціальна рекламна конструкція на фасаді буди...,1.0,6.000,0.356,2.1360,UA80000000000980793,Солом'янський
1,63984-17,2017-10-17,30403013,Чинний,2017-10-17,2026-02-04,Спеціальна рекламна конструкція на фасаді буди...,1.0,1.500,0.800,1.2000,UA80000000000210193,Дарницький
2,63894-17,2023-09-05,32386534,Чинний,2024-02-16,2029-02-15,Зупинковий комплекс та/або павільйон (лайтпостер),2.0,1.200,1.800,4.3200,UA80000000001078669,Шевченківський
3,56155-17,2017-07-07,34709124,Чинний,2017-07-07,2026-02-04,"Рекламна вивіска на будинку (будівлі), споруді",1.0,1.300,0.500,0.6500,UA80000000000126643,Голосіївський
4,56237-17-П-1,2024-02-19,41018978,Чинний,2024-08-14,2029-08-13,Конструкція на даху будинку (будівлі),1.0,6.082,2.313,14.0677,UA80000000000980793,Солом'янський
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15919,103654-25,2025-09-17,34729203,Чинний,2025-09-17,2030-09-16,"Щит, що стоїть окремо",2.0,6.000,3.000,36.0000,UA80000000000126643,Голосіївський
15920,103647-25,2025-09-17,34729203,Чинний,2025-09-17,2030-09-16,"Щит, що стоїть окремо",2.0,6.000,3.000,36.0000,UA80000000000126643,Голосіївський
15921,103660-25,2025-09-17,34729203,Чинний,2025-09-17,2030-09-16,"Щит, що стоїть окремо",2.0,6.000,3.000,36.0000,UA80000000000126643,Голосіївський
15922,103648-25,2025-09-17,34729203,Чинний,2025-09-17,2030-09-16,"Щит, що стоїть окремо",2.0,6.000,3.000,36.0000,UA80000000000126643,Голосіївський


In [118]:
distributor_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/distributor-info.csv")
distributor_info

,distributor,id
0,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «АПТЕК...,33549639
1,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «АВРОР...,30403013
2,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «БІГБОРД»,32386534
3,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «СІНЕВ...,34709124
4,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «КОСМО...,41018978
...,...,...
2254,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МЕРЕЖ...,41609173
2255,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ПАРІС...,43963505
2256,ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» (У ФОРМІ ТОВАРИ...,41960979
2257,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «СУШИ ...,37933258


In [119]:
#1 zadaniye
advertising_info.columns = advertising_info.columns.str.strip()
distributor_info.columns = distributor_info.columns.str.strip()

advertising_info.dropna(subset=["id", "distributorid"], inplace=True)
distributor_info.dropna(subset=["id"], inplace=True)

advertising_info.drop_duplicates(inplace=True)
distributor_info.drop_duplicates(inplace=True)

advertising_info['issuedDate'] = pd.to_datetime(advertising_info['issuedDate'], errors='coerce')
advertising_info['validFrom'] = pd.to_datetime(advertising_info['validFrom'], errors='coerce')
advertising_info['validThrough'] = pd.to_datetime(advertising_info['validThrough'], errors='coerce')
advertising_info['status'] = advertising_info['status'].str.lower().str.strip()
advertising_info['distributorid'] = advertising_info['distributorid'].astype(str)
distributor_info['id'] = distributor_info['id'].astype(str)

print(advertising_info.info())
print(distributor_info.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15924 entries, 0 to 15923
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              15924 non-null  object        
 1   issuedDate      15924 non-null  datetime64[ns]
 2   distributorid   15924 non-null  object        
 3   status          15924 non-null  object        
 4   validFrom       15924 non-null  datetime64[ns]
 5   validThrough    15924 non-null  datetime64[ns]
 6   type            15924 non-null  object        
 7   planesValue     15919 non-null  float64       
 8   horizontalSize  15913 non-null  float64       
 9   verticalSize    15913 non-null  float64       
 10  area            15924 non-null  float64       
 11  CATUTTC         15924 non-null  object        
 12  district        15924 non-null  object        
dtypes: datetime64[ns](3), float64(4), object(6)
memory usage: 1.6+ MB
None
<class 'pandas.core.frame.DataFrame

In [120]:
today = pd.Timestamp.today()
advertising_info["isActiveNow"] = (advertising_info["validFrom"]<=today)&(advertising_info["validThrough"]>=today)
print(advertising_info[["id", "validFrom","validThrough", "isActiveNow"]].head(10))
print(advertising_info["isActiveNow"].value_counts())

             id  validFrom validThrough  isActiveNow
0      63826-17 2017-10-30   2026-02-04         True
1      63984-17 2017-10-17   2026-02-04         True
2      63894-17 2024-02-16   2029-02-15         True
3      56155-17 2017-07-07   2026-02-04         True
4  56237-17-П-1 2024-08-14   2029-08-13         True
5      63895-17 2018-02-05   2026-02-04         True
6      63785-17 2018-02-05   2026-02-04         True
7      64001-17 2018-02-13   2026-02-04         True
8      64004-17 2018-02-01   2026-02-04         True
9      63827-17 2017-10-17   2026-02-04         True
isActiveNow
True     15631
False      293
Name: count, dtype: int64


In [121]:
!pip install ydata-profiling

In [122]:
import ydata_profiling
profile_adv = ydata_profiling.ProfileReport(advertising_info)
profile_adv.to_file("advertising_info_report.html")

profile_dist = ydata_profiling.ProfileReport(distributor_info)
profile_dist.to_file("distributor_info_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 14/14 [00:00<00:00, 36.99it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 29.49it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

2 zadaniye

In [123]:
#1
type_counts = advertising_info["type"].value_counts()
print(type_counts.head(5))

type
Спеціальна рекламна конструкція на фасаді будинку (будівлі)                               7358
Щит, що стоїть окремо                                                                     2774
Лайтпостер, що стоїть окремо                                                              1185
Спеціальна рекламна конструкція, що розміщується під кутом до фасаду будинку (будівлі)     977
Щит на фасаді будинку (будівлі)                                                            752
Name: count, dtype: int64


In [124]:
#2
pivo_type_district = advertising_info.pivot_table(index="district", columns="type", values="id", aggfunc="count", fill_value=0)
print(pivo_type_district)

type            Афішна (інформаційна) тумба  \
district                                      
Голосіївський                             0   
Дарницький                                0   
Деснянський                               0   
Дніпровський                              0   
Оболонський                               1   
Печерський                                2   
Подільський                               1   
Святошинський                             0   
Солом'янський                             0   
Шевченківський                            0   

type            Банер, панно на фасаді будинку (будівлі)  \
district                                                   
Голосіївський                                         17   
Дарницький                                            16   
Деснянський                                           13   
Дніпровський                                           6   
Оболонський                                           13   
Печерський     

In [125]:
#3
advertising_info = advertising_info.merge(distributor_info, left_on='distributorid', right_on='id', how='left')
advertising_info.rename(columns={'distributor': 'company'}, inplace=True)
company_counts = advertising_info['company'].value_counts()
print(company_counts)

company
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «БІГБОРД»                         956
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕКОСВІТ»                         466
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ - УКРАЇНА»                   422
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РА «ЛУВЕРС»                      411
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ПРАЙМ - ГРУП»                    361
                                                                         ... 
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МОБІЛАЙН ПЛАЗА»                    1
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «БОН ВОРЛД УКРАЇНА»                 1
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МЕДИЧНИЙ ЦЕНТР «НАША РОДИНА»       1
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ШАМС ГРУП»                         1
ТОВАРИСТВО З ДОДАТКОВОЮ ВІДПОВІДАЛЬНІСТЮ «УНІВЕРСАМ № 6 «ОЛІМПІЙСЬКИЙ»      1
Name: count, Length: 2257, dtype: int64


In [126]:
#4
company_area = advertising_info.groupby("company")["area"].sum().sort_values(ascending=False)
print(company_area.head(10))

company
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «СВІТ АУТДОР МЕДІА»    30052.9849
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «БІГБОРД»              17391.5056
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ТРИ О»                14437.1632
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РЕКЛЕН МЕДІА»         13410.6210
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РА «ЛУВЕРС»           10245.5887
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ОКТАГОН-АУТДОР»        9470.2634
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ - УКРАЇНА»         7987.9288
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ПРАЙМ - ГРУП»          7285.1462
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕПІЦЕНТР К»            7188.7692
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «АЙ ЕМ ЕЙ МЕДІА»        6811.5182
Name: area, dtype: float64


In [127]:
#5
pivot_area_district = advertising_info.pivot_table(index="district", columns="company", values="area", aggfunc="sum", fill_value=0)
print(pivot_area_district)

company         «ВСЕУКРАЇНСЬКИЙ ДУХОВНИЙ ЦЕНТР «ВІДРОДЖЕННЯ»  \
district                                                       
Голосіївський                                         31.625   
Дарницький                                             0.000   
Деснянський                                            0.000   
Дніпровський                                           0.000   
Оболонський                                            0.000   
Печерський                                             0.000   
Подільський                                            0.000   
Святошинський                                          0.000   
Солом'янський                                          0.000   
Шевченківський                                         0.000   

company         «НАЗЕЛСТАР» ПРИВАТНЕ ПІДПРИЄМСТВО З ІНОЗЕМНИМИ ІНВЕСТИЦІЯМИ  \
district                                                                      
Голосіївський                                              0.0000        

In [128]:
#6
today=pd.Timestamp.today()
closest_expiry = advertising_info.sort_values("validThrough").head(10)
print(closest_expiry[["company", "validThrough"]])

                                                 company validThrough
10917  ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕПІЦЕ...   2025-11-08
10889  ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «СІЛЬП...   2025-11-08
9466   ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ -...   2025-11-08
9467   ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ -...   2025-11-08
9468   ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ -...   2025-11-08
10226  АКЦІОНЕРНЕ ТОВАРИСТВО КОМЕРЦІЙНИЙ БАНК «ПРИВАТ...   2025-11-08
9007   ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕПІЦЕ...   2025-11-08
9008   ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕПІЦЕ...   2025-11-08
10200  ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ОКТАГ...   2025-11-08
9009   ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕПІЦЕ...   2025-11-08


In [130]:
#7
company_counts = advertising_info["company"].value_counts()
top5_companies = company_counts.head(5).index
top5_data=advertising_info[advertising_info["company"].isin(top5_companies)]
pivot_top5_type = top5_data.pivot_table(index="company", columns="type", aggfunc="size", fill_value=0)
print(pivot_top5_type)

type                                                Банер, панно на фасаді будинку (будівлі)  \
company                                                                                        
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «БІГБОРД»                                          2   
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕКОСВІТ»                                          0   
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ПРАЙМ ...                                         0   
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РА «ЛУ...                                         0   
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ - ...                                         0   

type                                                Елементи (частини) об'єктів благоустрою, що використовуються як рекламоносії (урна, лава тощо)  \
company                                                                                                                                              
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДП

In [133]:
top_type_per_company = pivot_top5_type.idxmax(axis=1)
print("Наиболее предпочтительный тип рекламы у топ5 компаний:")
print(top_type_per_company)

Наиболее предпочтительный тип рекламы у топ5 компаний:
company
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «БІГБОРД»                 Щит, що стоїть окремо
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ЕКОСВІТ»          Лайтпостер, що стоїть окремо
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ПРАЙМ - ГРУП»            Щит, що стоїть окремо
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РА «ЛУВЕРС»              Щит, що стоїть окремо
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «РТМ - УКРАЇНА»           Щит, що стоїть окремо
dtype: object
